<a href="https://colab.research.google.com/github/cagBRT/CUDA/blob/main/Nvidia_GPUs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Select the GPU to use by going to<br>

Runtime>Change Runtime Type><br>
Then select a GPU

This output provides information about the NVIDIA GPU that’s currently allocated to your Colab session.

In [14]:
!nvidia-smi

Fri Apr 12 19:33:23 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla V100-SXM2-16GB           Off | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0              27W / 300W |      0MiB / 16384MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

## Google Colab’s Pros and Cons<br>
**Pros**<br>
Free Access to GPUs: Google Colab provides free access to GPUs, making it an attractive platform for data scientists and software engineers who want to leverage powerful computing resources without incurring additional costs.<br>

Cloud-Based Platform: Being a cloud-based platform, Google Colab eliminates the need for users to set up and maintain their own computing infrastructure. This allows for easier collaboration and seamless sharing of Jupyter notebooks.<br>

Ease of Use: Colab is known for its user-friendly interface and seamless integration with Jupyter notebooks. Users can quickly set up and run machine learning experiments without the hassle of configuring hardware or software.<br>

Wide Adoption: Due to its free GPU access and ease of use, Google Colab has gained widespread adoption among the machine learning community. This popularity fosters a supportive user community and ensures that users can find resources and help easily.

**Cons**<br>
Variable GPU Specs: The allocated GPU specifications in Google Colab can vary, and users may not always have clarity on the available resources. This variability can impact the performance and efficiency of machine learning experiments.<br>

Limited GPU Memory: The allocated GPU memory may not always be sufficient for complex machine learning models or large datasets. Users may face limitations that hinder their ability to train certain types of models or handle extensive datasets.<br>

Resource Availability: Larger GPUs may not always be available, and requesting a change in GPU type or size might result in a longer wait time before the Colab session is ready. This can be a drawback, especially when time is crucial for the experimentation process.



## Error Handling
**Unavailable GPUs**: Users should be prepared to handle situations where GPUs are not available due to high demand or maintenance. It’s advisable to have contingency plans, such as trying the experiment at a later time or exploring alternative platforms.

**Error in GPU Allocation**: If there is an error in GPU allocation, users should check their Colab runtime settings and ensure that they have selected the correct GPU type. Clear instructions on troubleshooting such errors can help users quickly resolve issues.

**Memory Exhaustion**: In cases where GPU memory is insufficient, users should consider optimizing their machine learning models, utilizing data batching techniques, or exploring alternatives like distributed computing. Clear guidance on managing memory issues can enhance the user experience.

**Long Wait Times**: Users requesting larger GPUs should be aware of potential longer wait times. Adequate communication about expected wait times can help manage user expectations and allow them to plan their work accordingly.



Google Colab has a new paid tier, [Pay As You Go](https://colab.research.google.com/signup), giving anyone the option to purchase additional compute time in Colab with or without a paid subscription. This grants access to Colab’s  powerful NVIDIA GPUs and gives you more control over your machine learning environment.

In [15]:
# To show that if there is cuda tookit installed
!ls /usr/local

bin    cuda	cuda-12.2  games	       include	lib64	   man	 share
colab  cuda-12	etc	   _gcs_config_ops.so  lib	licensing  sbin  src


In [16]:
# To show that if we have the nvcc command
!which nvcc

In [17]:
# To show the property of the nvidia card(On my one, I use the K80)
!nvidia-smi

Fri Apr 12 19:33:23 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla V100-SXM2-16GB           Off | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0              27W / 300W |      0MiB / 16384MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [18]:
# Write a cu file contain the host and kernel code
%%writefile hello.cu

#include <cstdio>

__global__ void hello(void)
{
  printf("GPU: Hello!\n");
}

int main(int argc,char **argv)
{
  printf("CPU: Hello!\n");
  hello<<<1,10>>>();
  cudaDeviceReset();
  return 0;
}

Overwriting hello.cu


In [19]:
#!nvcc --help

In [20]:
!nvcc --list-gpu-arch

/bin/bash: line 1: nvcc: command not found


In [21]:
# Compile the code. The flag is needed if you use the Tesla K80.
!nvcc -arch=sm_50 -gencode=arch=compute_50,code=sm_50 hello.cu -o hello

/bin/bash: line 1: nvcc: command not found


In [22]:
!./hello

CPU: Hello!
GPU: Hello!
GPU: Hello!
GPU: Hello!
GPU: Hello!
GPU: Hello!
GPU: Hello!
GPU: Hello!
GPU: Hello!
GPU: Hello!
GPU: Hello!
